In [29]:
import pandas as pd
import numpy as np
import sklearn.metrics.pairwise as pw
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from tabulate import tabulate

In [30]:
vinhos = pd.read_csv(r'banco_de_dados_vinhos.csv')

In [34]:
vinhos.sample(5)

,Vinho,País de origem,Tipo de uva,Doçura,Acidez,Taninos,Porcentagem de álcool,Cor,Corpo,Aroma,Harmonização
22,Tenuta San Guido Guidalberto,Itália,"Cabernet Sauvignon, Merlot",Seco,Média a alta,Moderados,13.5,Rubi claro,Médio a encorpado,"Frutas vermelhas, Ervas secas, Tabaco, Especia...","Massas com molho de carne, Pratos de carne ass..."
0,Château Margaux (Pavillon Rouge),França,"Cabernet Sauvignon, Merlot",Seco,Alta,Elevados,13.5,Rubi profundo,Encorpado,"Frutas vermelhas, Cassis, Cedro, Notas terrosas","Carnes vermelhas, Cordeiro, Queijos maturados"
9,Quinta do Crasto Touriga Nacional,Portugal,Touriga Nacional,Seco,Média,Moderados,13.5,Rubi intenso,Encorpado,"Violetas, Frutas escuras, Especiarias","Carnes de caça, Assados, Queijos curados"
2,Antinori Tignanello,Itália,"Sangiovese, Cabernet Sauvignon",Seco,Alta,Moderados,13.5,Rubi intenso,Médio a encorpado,"Cerejas, Couro, Tabaco, Baunilha","Massas com molho de tomate, Carnes assadas, Qu..."
27,Paul Jaboulet Aîné Hermitage La Chapelle,França,Syrah,Seco,Alta,Elevados,14.5,Rubi profundo,Encorpado,"Frutas negras, Pimenta, Especiarias, Carvalho","Carnes de caça, Cordeiro, Queijos fortes"


In [33]:
vinhos['Porcentagem de álcool'] = vinhos['Porcentagem de álcool'].apply(lambda x: str(x))

In [41]:
# Cria o vetorizador TF-IDF
vetorizador = TfidfVectorizer()
# Ajusta e transforma os dados da coluna 'Harmonização' em uma matriz TF-IDF
tfidf = vetorizador.fit_transform(vinhos['Harmonização'])
# Calcula a similaridade do cosseno
similaridade = cosine_similarity(tfidf)
# Cria um DataFrame com os resultados da similaridade, usando a coluna 'Vinho' como referência
sim_vinhos = pd.DataFrame(similaridade, columns=vinhos['Vinho'], index=vinhos['Vinho'])


# Função da Similaridade e do print com os resultados em formato de tabela
def escolhervinho(x, final_df):
    resultado = pd.DataFrame(sim_vinhos[x].sort_values(ascending=False))
    resultado.columns = ['Harmonização']
    data = [[i, nome, f"{valor['Harmonização'] * 100:.2f}%"]
            for i, (nome, valor) in enumerate(resultado.iloc[1:6].iterrows(), 1)]
    print("=" * 55)
    print(f"Os vinhos mais parecidos com '{x}' são:")
    print(tabulate(data, headers=["Opção", "Vinho", "Similaridade"], tablefmt="fancy_grid"))


# Função para criar um ciclo que pergunta se o cliente deseja verificar outro vinho ou finalizar o programa
def loop():
    while True:
        print("=" * 55)
        y = input("Deseja ver outro vinho? S/N: ").strip().upper()
        if y == "S":
            x = input("Digite o nome do vinho: ").strip()
            while True:
                if x in sim_vinhos.columns:
                    final_df = pd.DataFrame(sim_vinhos[x].sort_values(ascending=False))
                    final_df.columns = ['Harmonização']
                    escolhervinho(x, final_df)
                    break
                else:
                    print("Vinho não encontrado. Digite novamente.")
                    x = input("Digite o nome do vinho: ").strip()
        elif y == "N":
            print("Programa Finalizado!")
            break
        else:
            print("Opção Inválida!")


# Execução inicial com escolha e verificação do vinho
x = input("Digite o nome do vinho: ").strip()
while True:
    if x in sim_vinhos.columns:
        final_df = pd.DataFrame(sim_vinhos[x].sort_values(ascending=False))
        final_df.columns = ['Harmonização']
        escolhervinho(x, final_df)
        break
    else:
        print("Vinho não encontrado. Digite novamente.")
        x = input("Digite o nome do vinho: ").strip()


loop()

Digite o nome do vinho: Opus One
Os vinhos mais parecidos com 'Opus One' são:
╒═════════╤═══════════════════════════╤════════════════╕
│   Opção │ Vinho                     │ Similaridade   │
╞═════════╪═══════════════════════════╪════════════════╡
│       1 │ Harlan Estate             │ 81.80%         │
├─────────┼───────────────────────────┼────────────────┤
│       2 │ Caymus Cabernet Sauvignon │ 61.89%         │
├─────────┼───────────────────────────┼────────────────┤
│       3 │ Château Lafite Rothschild │ 50.08%         │
├─────────┼───────────────────────────┼────────────────┤
│       4 │ Quinta do Vale Meão       │ 46.45%         │
├─────────┼───────────────────────────┼────────────────┤
│       5 │ Marqués de Riscal Reserva │ 46.41%         │
╘═════════╧═══════════════════════════╧════════════════╛
Deseja ver outro vinho? S/N: s
Digite o nome do vinho: Quinta do Crasto Touriga Nacional	
Os vinhos mais parecidos com 'Quinta do Crasto Touriga Nacional' são:
╒═════════╤══════════